# `Business Problem`

## Import Packages

In [124]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='whitegrid')
import warnings 
warnings.filterwarnings('ignore')
from datasist.structdata import detect_outliers
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import plotly.express as px
import plotly.figure_factory as ff
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder , MinMaxScaler , RobustScaler
from category_encoders import BinaryEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from imblearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score , f1_score
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import GridSearchCV
import joblib as jb

## Read Data

In [125]:
df = pd.read_csv('train_ctrUa4K.csv')
df.sample(10)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
109,LP001384,Male,Yes,3+,Not Graduate,No,2071,754.0,94.0,480.0,1.0,Semiurban,Y
271,LP001891,Male,Yes,0,Graduate,No,11146,0.0,136.0,360.0,1.0,Urban,Y
66,LP001228,Male,No,0,Not Graduate,No,3200,2254.0,126.0,180.0,0.0,Urban,N
521,LP002690,Male,No,0,Graduate,No,2500,0.0,55.0,360.0,1.0,Semiurban,Y
116,LP001404,Female,Yes,0,Graduate,No,3167,2283.0,154.0,360.0,1.0,Semiurban,Y
410,LP002318,Female,No,1,Not Graduate,Yes,3867,0.0,62.0,360.0,1.0,Semiurban,N
483,LP002541,Male,Yes,0,Graduate,No,10833,0.0,234.0,360.0,1.0,Semiurban,Y
346,LP002130,Male,Yes,NaN,Not Graduate,No,3523,3230.0,152.0,360.0,0.0,Rural,N
319,LP002050,Male,Yes,1,Graduate,Yes,10000,0.0,155.0,360.0,1.0,Rural,N
406,LP002308,Male,Yes,0,Not Graduate,No,2167,2400.0,115.0,360.0,1.0,Urban,Y


# 1. EDA

### 1.1 Investegation 

In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [127]:
# Check Duplicated records
df.duplicated().sum()

0

In [128]:
# check null values in data
df.isna().sum()/df.shape[0]

Loan_ID              0.000000
Gender               0.021173
Married              0.004886
Dependents           0.024430
Education            0.000000
Self_Employed        0.052117
ApplicantIncome      0.000000
CoapplicantIncome    0.000000
LoanAmount           0.035831
Loan_Amount_Term     0.022801
Credit_History       0.081433
Property_Area        0.000000
Loan_Status          0.000000
dtype: float64

In [129]:
# Check Count of unique values in each feature
for col in df.columns:
    print ((col.ljust(20)), '=>' , (df[col].nunique()))

Loan_ID              => 614
Gender               => 2
Married              => 2
Dependents           => 4
Education            => 2
Self_Employed        => 2
ApplicantIncome      => 505
CoapplicantIncome    => 287
LoanAmount           => 203
Loan_Amount_Term     => 10
Credit_History       => 2
Property_Area        => 3
Loan_Status          => 2


In [131]:
df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


#### Notes 
- Most of Data's Columns have Nan Values (so, we have 2 solution.) 
    - remove it or fill it. 
- No duplicated rows founded 

###  1.2 Working with data issues 

## 2.Business Question

### what is the max income of applicant?

In [62]:
df[df['ApplicantIncome'] == 81000]

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
409,Male,Yes,3,Graduate,No,81000,0.0,360.0,360.0,0.0,Rural,0


### what is the max amount of loan request?

In [63]:
df[df['LoanAmount'] == 700]

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
171,NaN,Yes,3,Graduate,No,51763,0.0,700.0,300.0,1.0,Urban,1


### 1.2.1 Cleaning

In [132]:
df.drop('Loan_ID',axis=1,inplace=True)

In [133]:
# Remove + sign in +3 in Dependents 
df['Dependents']=df['Dependents'].apply(lambda x:str(x).replace('3+','3'))

### 1.2.2 Checking Outliers

In [134]:
fig = px.box(df,x='LoanAmount')
fig.show()

In [135]:
fig = px.box(df,x='ApplicantIncome')
fig.show()

In [136]:
fig = px.box(df,x='CoapplicantIncome')
fig.show()

In [138]:
# Handling outliers
Outlier_Cols=['ApplicantIncome','CoapplicantIncome','LoanAmount']

for col in Outlier_Cols:
    outliers_indices = detect_outliers(df, 0, [col])
    col_median = df[col].median()
    df[col].iloc[outliers_indices] = col_median

In [139]:
# Check after handling
fig = px.box(df,x='ApplicantIncome')
fig.show()

### 1.2.3Label Encoding for Category columns

In [140]:
# Apply Label Encoding for Category columns
Lab_Encoding_Cols=['Gender','Married','Education','Self_Employed','Loan_Status']

for col in Lab_Encoding_Cols:
    df[col] = LabelEncoder().fit_transform(df[col].values)
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,0,0,0,0,5849.0,0.0,NaN,360.0,1.0,Urban,1
1,1,1,1,0,0,4583.0,1508.0,128.0,360.0,1.0,Rural,0
2,1,1,0,0,1,3000.0,0.0,66.0,360.0,1.0,Urban,1
3,1,1,0,1,0,2583.0,2358.0,120.0,360.0,1.0,Urban,1
4,1,0,0,0,0,6000.0,0.0,141.0,360.0,1.0,Urban,1


> let's build our Prediction Model

## 3.Preprocessing & Modeling

In [141]:
# split it into x--> data,  y--> target
X = df.drop("Loan_Status", axis =1)
y = df['Loan_Status']

In [142]:
# Check if data imbalanced
y.value_counts()*100/len(y)

1    68.729642
0    31.270358
Name: Loan_Status, dtype: float64

In [143]:
## Models 
models = [
    ('logestic Regression', LogisticRegression()),
    ('Naive Bayes', GaussianNB()),
    ('KNN', KNeighborsClassifier(n_neighbors=5)),
    ('DTC', DecisionTreeClassifier()), 
    ('RFC', RandomForestClassifier()), 
    ('ABC', AdaBoostClassifier()), 
    ('GBC', GradientBoostingClassifier()), 
    ('XGBC', XGBClassifier())
]

In [120]:
## check the unique values of each feature 
for col in df.columns:
    print (col)
    print(df[col].unique())
    print(50*'*')

Gender
[1 0 2]
**************************************************
Married
[0 1 2]
**************************************************
Dependents
['0' '1' '2' '3' 'nan']
**************************************************
Education
[0 1]
**************************************************
Self_Employed
[0 1 2]
**************************************************
ApplicantIncome
[ 5849  4583  3000  2583  6000  5417  2333  3036  4006 12841  3200  2500
  3073  1853  1299  4950  3596  3510  4887  2600  7660  5955  3365  3717
  9560  2799  4226  1442  3750  4166  3167  4692  3500 12500  2275  1828
  3667  3748  3600  1800  2400  3941  4695  3410  5649  5821  2645  4000
  1928  3086  4230  4616 11500  2708  2132  3366  8080  3357  3029  2609
  4945  5726 10750  7100  4300  3208  1875  4755  5266  1000  3333  3846
  2395  1378  3988  2366  8566  5695  2958  6250  3273  4133  3620  6782
  2484  1977  4188  1759  4288  4843 13650  4652  3816  3052 11417  7333
  3800  2071  5316  2929  3572  7451  505

In [121]:
Nan_C_Cols=['Gender','Married','Self_Employed']
Nan_N_Cols=['Dependents','LoanAmount','Loan_Amount_Term','Credit_History']
Outlier_Cols=['ApplicantIncome','CoapplicantIncome','LoanAmount']
Bin_Encoding_Cols=['Property_Area']


# Initialize StandardScaler
scaler = StandardScaler()

# Initialize SMOTE TO Oversample
Oversampling = SMOTE()

# Create the ColumnTransformer
preprocessor = ColumnTransformer(
transformers=[
    ('MissVaC', SimpleImputer(strategy='most_frequent'), Nan_C_Cols), # Replace missing values with mode
    ('MissVaN', SimpleImputer(strategy='median'), Nan_N_Cols), # Replace missing values with median
    ('binarycols', BinaryEncoder(), Bin_Encoding_Cols)] # Encoder for Categorical columns by BinaryEncoder
            , remainder='passthrough') 

for model_name, model in models:
    # print model name
    print(f"For {model_name}:")
    # Steps Creation
    steps = list()

    steps.append(('preprocessor', preprocessor))
    steps.append(('scaler', scaler))
    steps.append(('Oversampling', Oversampling))
    steps.append((model_name, model))

    # Create the pipeline
    pipeline = Pipeline(steps=steps)


    ## scoring
    #Perform cross-validation
    # Example evaluation metrics
    cv_results = cross_validate(pipeline, X, y, scoring='accuracy', cv=5, error_score='raise', return_train_score=True)  # X and y are your input data and targets

    # Access the results
    print("test :", cv_results['test_score'].mean())
    print("train:", cv_results['train_score'].mean())
    print('*' * 50)
    print('*' * 100)
    print('*' * 100)

For logestic Regression:
test : 0.7425829668132746
train: 0.7805358236881758
**************************************************
****************************************************************************************************
****************************************************************************************************
For Naive Bayes:
test : 0.7964814074370252
train: 0.8078179590349874
**************************************************
****************************************************************************************************
****************************************************************************************************
For KNN:
test : 0.6872717579634813
train: 0.8236972827976752
**************************************************
****************************************************************************************************
****************************************************************************************************
For DTC:
test : 0.6970811675329868

> ### Let's Try after handling Outliers

In [144]:
Nan_C_Cols=['Gender','Married','Self_Employed']
Nan_N_Cols=['Dependents','LoanAmount','Loan_Amount_Term','Credit_History']
Outlier_Cols=['ApplicantIncome','CoapplicantIncome','LoanAmount']
Bin_Encoding_Cols=['Property_Area']


# Initialize StandardScaler
scaler = StandardScaler()

# Initialize SMOTE TO Oversample
Oversampling = SMOTE()

# Create the ColumnTransformer
preprocessor = ColumnTransformer(
transformers=[
    ('MissVaC', SimpleImputer(strategy='most_frequent'), Nan_C_Cols), # Replace missing values with mode
    ('MissVaN', SimpleImputer(strategy='median'), Nan_N_Cols), # Replace missing values with median
    ('binarycols', BinaryEncoder(), Bin_Encoding_Cols)] # Encoder for Categorical columns by BinaryEncoder
            , remainder='passthrough') 

for model_name, model in models:
    # print model name
    print(f"For {model_name}:")
    # Steps Creation
    steps = list()

    steps.append(('preprocessor', preprocessor))
    steps.append(('scaler', scaler))
    steps.append(('Oversampling', Oversampling))
    steps.append((model_name, model))

    # Create the pipeline
    pipeline = Pipeline(steps=steps)


    ## scoring
    #Perform cross-validation
    # Example evaluation metrics
    cv_results = cross_validate(pipeline, X, y, scoring='accuracy', cv=5, error_score='raise', return_train_score=True)  # X and y are your input data and targets

    # Access the results
    print("test :", cv_results['test_score'].mean())
    print("train:", cv_results['train_score'].mean())
    print('*' * 50)
    print('*' * 100)
    print('*' * 100)

For logestic Regression:
test : 0.7328535252565641
train: 0.7715861109731261
**************************************************
****************************************************************************************************
****************************************************************************************************
For Naive Bayes:
test : 0.8062108489937359
train: 0.8086342788071466
**************************************************
****************************************************************************************************
****************************************************************************************************
For KNN:
test : 0.6790883646541384
train: 0.828995910122034
**************************************************
****************************************************************************************************
****************************************************************************************************
For DTC:
test : 0.6937758230041318


> Accuracy After handling outliers is better

> ### Let's try after Feature Selection 

In [145]:
Nan_C_Cols=['Gender','Married','Self_Employed']
Nan_N_Cols=['Dependents','LoanAmount','Loan_Amount_Term','Credit_History']
Outlier_Cols=['ApplicantIncome','CoapplicantIncome','LoanAmount']
Bin_Encoding_Cols=['Property_Area']


# Initialize StandardScaler
scaler = StandardScaler()

# Initialize SMOTE TO Oversample
Oversampling = SMOTE()

# Initialize RFE feature selector
RFE_selector = RFE(XGBClassifier(), n_features_to_select=7)   

# Create the ColumnTransformer
preprocessor = ColumnTransformer(
transformers=[
    ('MissVaC', SimpleImputer(strategy='most_frequent'), Nan_C_Cols), # Replace missing values with mode
    ('MissVaN', SimpleImputer(strategy='median'), Nan_N_Cols), # Replace missing values with median
    ('binarycols', BinaryEncoder(), Bin_Encoding_Cols)] # Encoder for Categorical columns by BinaryEncoder
            , remainder='passthrough') 

for model_name, model in models:
    # print model name
    print(f"For {model_name}:")
    # Steps Creation
    steps = list()

    steps.append(('preprocessor', preprocessor))
    steps.append(('scaler', scaler))
    steps.append(('Oversampling', Oversampling))
    steps.append(('feature_selector', RFE_selector))  # RFE feature selection
    steps.append((model_name, model))

    # Create the pipeline
    pipeline = Pipeline(steps=steps)


    ## scoring
    #Perform cross-validation
    # Example evaluation metrics
    cv_results = cross_validate(pipeline, X, y, scoring='accuracy', cv=5, error_score='raise', return_train_score=True)  # X and y are your input data and targets

    # Access the results
    print("test :", cv_results['test_score'].mean())
    print("train:", cv_results['train_score'].mean())
    print('*' * 50)
    print('*' * 100)
    print('*' * 100)

For logestic Regression:
test : 0.7476209516193523
train: 0.7703558359412515
**************************************************
****************************************************************************************************
****************************************************************************************************
For Naive Bayes:
test : 0.8078368652538984
train: 0.810261950888348
**************************************************
****************************************************************************************************
****************************************************************************************************
For KNN:
test : 0.716566706650673
train: 0.786236815524978
**************************************************
****************************************************************************************************
****************************************************************************************************
For DTC:
test : 0.7312541650006663
tr

In [147]:
# Nan_C_Cols=['Gender','Married','Self_Employed']
# Nan_N_Cols=['Dependents','LoanAmount','Loan_Amount_Term','Credit_History']
# Outlier_Cols=['ApplicantIncome','CoapplicantIncome','LoanAmount']
# Lab_Encoding_Cols=['Gender','Married','Education','Self_Employed']
# Bin_Encoding_Cols=['Property_Area']

# # Custom function to handle outliers using IQR and replace with median
# def handle_outliers_iqr_median(column):
#     Q1 = column.quantile(0.25)
#     Q3 = column.quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     return column.apply(lambda x: np.where((x < lower_bound) | (x > upper_bound), np.median(column), x))

# # Create a transformer using FunctionTransformer
# outlier_transformer = FunctionTransformer(handle_outliers_iqr_median)

# # Initialize RFE feature selector
# RFE_selector = RFE(XGBClassifier(), n_features_to_select=7)   


# # Initialize StandardScaler
# scaler = StandardScaler()

# # Initialize SMOTE TO Oversample
# Oversampling = SMOTE()

# # Create the ColumnTransformer
# preprocessor = ColumnTransformer(
# transformers=[
#     ('MissVaC', SimpleImputer(strategy='most_frequent'), Nan_C_Cols), # Replace missing values with mode
#     ('MissVaN', SimpleImputer(strategy='median'), Nan_N_Cols), # Replace missing values with median
#     ('outliers', outlier_transformer, Outlier_Cols), # handle outliers using IQR and replace with median
#     ('Label_Enc',LabelEncoder(),Lab_Encoding_Cols),  # Encode Categorical Columns by Label Encoder      
#     ('binarycols', BinaryEncoder(), Bin_Encoding_Cols)] # Encoder for Categorical columns by BinaryEncoder
#             , remainder='passthrough') 

# for model_name, model in models:
#     # print model name
#     print(f"For {model_name}:")
#     # Steps Creation
#     steps = list()

#     steps.append(('preprocessor', preprocessor))
#     steps.append(('scaler', scaler))
#     steps.append(('Oversampling', Oversampling))
#     steps.append(('feature_selector', RFE_selector))  # RFE feature selection
#     steps.append((model_name, model))

#     # Create the pipeline
#     pipeline = Pipeline(steps=steps)


#     ## scoring
#     #Perform cross-validation
#     # Example evaluation metrics
#     cv_results = cross_validate(pipeline, X, y, scoring='accuracy', cv=5, error_score='raise', return_train_score=True)  # X and y are your input data and targets

#     # Access the results
#     print("test :", cv_results['test_score'].mean())
#     print("train:", cv_results['train_score'].mean())
#     print('*' * 50)
#     print('*' * 100)
#     print('*' * 100)

> ### Naive Bayes then RandomForestClassifier is best Model

> #### lets try Hyperparameters Tuning for Random forest and Naive bayes and watch the Result

In [148]:
# Define your parameter grid for GridSearchCV
param_grid_rfc = {
   'RFC__bootstrap': [True],
    'RFC__max_depth': [80, 90, 100, 110],
    'RFC__max_features': [2, 3],
    'RFC__min_samples_leaf': [3, 4, 5],
    'RFC__min_samples_split': [8, 10, 12],
    'RFC__n_estimators': [100, 200, 300, 1000]
}

In [149]:
steps=[]
steps.append(('preprocessor', preprocessor))
steps.append(('scaler', scaler))
steps.append(('Oversampling', Oversampling))
steps.append(('feature_selector', RFE_selector))
steps.append(('RFC',RandomForestClassifier()))
pipeline=Pipeline(steps=steps)

In [151]:
# Create GridSearchCV instance
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid_rfc, cv=5, scoring='accuracy', n_jobs=-1, return_train_score=True)

# Fit the pipeline with GridSearch to the data
grid_search.fit(X, y)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'RFC__bootstrap': True, 'RFC__max_depth': 110, 'RFC__max_features': 2, 'RFC__min_samples_leaf': 3, 'RFC__min_samples_split': 10, 'RFC__n_estimators': 100}
Best Score: 0.79156337465014


In [152]:
# Get the mean test score and mean train score for the best estimator
mean_test_score = grid_search.cv_results_['mean_test_score'][grid_search.best_index_]
mean_train_score = grid_search.cv_results_['mean_train_score'][grid_search.best_index_]

print("Mean Test Score:", mean_test_score)
print("Mean Train Score:", mean_train_score)

Mean Test Score: 0.79156337465014
Mean Train Score: 0.8151491066845494


> ##### Lets try Naive Bayes

In [155]:
# Define the hyperparameter grid
param_grid_nb = {'Naive Bayes__var_smoothing': np.logspace(0, -9, num=100)}

In [156]:
steps=[]
steps.append(('preprocessor', preprocessor))
steps.append(('scaler', scaler))
steps.append(('Oversampling', Oversampling))
steps.append(('feature_selector', RFE_selector))
steps.append(('Naive Bayes',GaussianNB()))
pipeline=Pipeline(steps=steps)

In [157]:
# Create GridSearchCV instance
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid_nb, cv=5, scoring='accuracy', n_jobs=-1, return_train_score=True)

# Fit the pipeline with GridSearch to the data
grid_search.fit(X, y)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'Naive Bayes__var_smoothing': 0.03511191734215131}
Best Score: 0.8111022257763562


In [158]:
# Get the mean test score and mean train score for the best estimator
mean_test_score = grid_search.cv_results_['mean_test_score'][grid_search.best_index_]
mean_train_score = grid_search.cv_results_['mean_train_score'][grid_search.best_index_]

print("Mean Test Score:", mean_test_score)
print("Mean Train Score:", mean_train_score)

Mean Test Score: 0.8111022257763562
Mean Train Score: 0.8098546189127879


> ## Naive Bayes Better

In [159]:
final_model=grid_search.best_estimator_
final_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('MissVaC',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['Gender', 'Married',
                                                   'Self_Employed']),
                                                 ('MissVaN',
                                                  SimpleImputer(strategy='median'),
                                                  ['Dependents', 'LoanAmount',
                                                   'Loan_Amount_Term',
                                                   'Credit_History']),
                                                 ('binarycols', BinaryEncoder(),
                                                  ['Property_Area'])])),
                ('scaler', St...
                                             learning_rate=None, max_bin=None,
                                             max_cat_threshold=None,
                                             max_cat_to_onehot=None,
                                             max_delta_step=None,
                                             max_depth=None, max_leaves=None,
                                             min_child_weight=None, missing=nan,
                                             monotone_constraints=None,
                                             n_estimators=100, n_jobs=None,
                                             num_parallel_tree=None,
                                             predictor=None, random_state=None, ...),
                     n_features_to_select=7)),
                ('Naive Bayes', GaussianNB(var_smoothing=0.03511191734215131))])

In [160]:
jb.dump(final_model,"Model_Loan_Det_Final.pkl")
jb.dump(X.columns,"Inputs_Loan_Det_Final.pkl")

['Inputs_Loan_Det_Final.pkl']

In [161]:
for col in df.columns:
    print(f"test_df.loc[0,'{col}']= {col}")

test_df.loc[0,'Gender']= Gender
test_df.loc[0,'Married']= Married
test_df.loc[0,'Dependents']= Dependents
test_df.loc[0,'Education']= Education
test_df.loc[0,'Self_Employed']= Self_Employed
test_df.loc[0,'ApplicantIncome']= ApplicantIncome
test_df.loc[0,'CoapplicantIncome']= CoapplicantIncome
test_df.loc[0,'LoanAmount']= LoanAmount
test_df.loc[0,'Loan_Amount_Term']= Loan_Amount_Term
test_df.loc[0,'Credit_History']= Credit_History
test_df.loc[0,'Property_Area']= Property_Area
test_df.loc[0,'Loan_Status']= Loan_Status


In [130]:
## check the unique values of each feature 
for col in df.columns:
    print (col)
    print(df[col].unique())
    print(50*'*')

Loan_ID
['LP001002' 'LP001003' 'LP001005' 'LP001006' 'LP001008' 'LP001011'
 'LP001013' 'LP001014' 'LP001018' 'LP001020' 'LP001024' 'LP001027'
 'LP001028' 'LP001029' 'LP001030' 'LP001032' 'LP001034' 'LP001036'
 'LP001038' 'LP001041' 'LP001043' 'LP001046' 'LP001047' 'LP001050'
 'LP001052' 'LP001066' 'LP001068' 'LP001073' 'LP001086' 'LP001087'
 'LP001091' 'LP001095' 'LP001097' 'LP001098' 'LP001100' 'LP001106'
 'LP001109' 'LP001112' 'LP001114' 'LP001116' 'LP001119' 'LP001120'
 'LP001123' 'LP001131' 'LP001136' 'LP001137' 'LP001138' 'LP001144'
 'LP001146' 'LP001151' 'LP001155' 'LP001157' 'LP001164' 'LP001179'
 'LP001186' 'LP001194' 'LP001195' 'LP001197' 'LP001198' 'LP001199'
 'LP001205' 'LP001206' 'LP001207' 'LP001213' 'LP001222' 'LP001225'
 'LP001228' 'LP001233' 'LP001238' 'LP001241' 'LP001243' 'LP001245'
 'LP001248' 'LP001250' 'LP001253' 'LP001255' 'LP001256' 'LP001259'
 'LP001263' 'LP001264' 'LP001265' 'LP001266' 'LP001267' 'LP001273'
 'LP001275' 'LP001279' 'LP001280' 'LP001282' 'LP001289

In [166]:
%%writefile streamlit_app_Loan_Det_final.py

import joblib
import streamlit as st
import category_encoders
import pandas as pd
import sklearn

Model= joblib.load("Model_Loan_Det_Final.pkl")
Inputs= joblib.load("Inputs_Loan_Det_Final.pkl")

def Gend(Gender):
    if Gender == 'Female':
        return int(0)
    else:
        return int(1)
    
def Marr(Married):
    if Married == 'No':
        return int(0)
    else:
        return int(1)
    
def Dep(Dependents):
    if Dependents == '3+':
        return int(3)
    else:
        return int(Dependents)

def Educ(Education):
    if Education == 'Graduate':
        return int(0)
    else:
        return int(1)

def Emp(Self_Employed):
    if Self_Employed == 'No':
        return int(0)
    else:
        return int(1)

def prediction(Gender,Married, Dependents, Education, Self_Employed, ApplicantIncome, CoapplicantIncome, 
               LoanAmount, Loan_Amount_Term, Credit_History, Property_Area ):
    gender=Gend(Gender)
    married=Marr(Married)
    dependents=Dep(Dependents)
    education=Educ(Education)
    employed=Emp(Self_Employed)
    test_df=pd.DataFrame(columns=Inputs)
    test_df.loc[0,'Gender']= gender
    test_df.loc[0,'Married']= married
    test_df.loc[0,'Dependents']= dependents
    test_df.loc[0,'Education']= education
    test_df.loc[0,'Self_Employed']= employed
    test_df.loc[0,'ApplicantIncome']= ApplicantIncome
    test_df.loc[0,'CoapplicantIncome']= CoapplicantIncome
    test_df.loc[0,'LoanAmount']= LoanAmount
    test_df.loc[0,'Loan_Amount_Term']= Loan_Amount_Term
    test_df.loc[0,'Credit_History']= Credit_History
    test_df.loc[0,'Property_Area']= Property_Area
    
    result= Model.predict(test_df)
    if result[0]== 0 :
        return st.text("Please,Don't Accept on Loan")
    else:
         return st.text("Please, Accept on Loan") 
        
def main():
    
    ## Setting up the page title
    st.set_page_config(page_title= 'Loan Detection Prediction')
    
     # Add a title in the middle of the page using Markdown and CSS
    st.markdown("<h1 style='text-align: center;text-decoration: underline;color:GoldenRod'>Loan Detection Prediction</h1>", unsafe_allow_html=True)
    
    Gender=st.selectbox('Insert Gender',  ['Male','Female'])
    
    Married=st.selectbox('Are you married?',['Yes','No'])
    
    Dependents=st.selectbox('Select how many Dependents', ['0' ,'1' ,'2' ,'3+'])
    
    Education =st.selectbox('Select Education', ['Graduate','Not Graduate'])

    Self_Employed =st.selectbox('Are you employed', ['Yes', 'No'])

    ApplicantIncome=st.number_input('Insert ApplicantIncome in Thousands',min_value=100, max_value=30000, value=5000,step=500)
    
    CoapplicantIncome=st.number_input('Insert CoapplicantIncome in Thousands',min_value=0, max_value=40000, value=5000,step=500)
    
    LoanAmount=st.number_input('Insert LoanAmount in Thousands',min_value=9, max_value=1000, value=500,step=10)

    Loan_Amount_Term=st.slider('Choose Loan Amount Term in Months', min_value=10, max_value=400, value=12,step=10)

    Credit_History=st.radio(' Insert Credit_History', ['1', '0'])

    Property_Area=st.selectbox('Select Property Area', ['Urban', 'Rural', 'Semiurban'])
  
    
    if st.button('predict'):
        results= prediction(Gender,Married, Dependents, Education, Self_Employed, ApplicantIncome, CoapplicantIncome, 
               LoanAmount, Loan_Amount_Term, Credit_History, Property_Area )

if __name__ == '__main__':
    main()

Overwriting streamlit_app_Loan_Det_final.py


In [167]:
!streamlit run streamlit_app_Loan_Det_final.py

^C
